In [205]:
import pandas as pd
import shutil
from pathlib import Path
from tqdm import tqdm
import docx
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

In [206]:
file_path = Path("填写好的问卷/External Team Members-T3合并.docx")
doc = Document(file_path)
from docx.oxml.shared import OxmlElement
for table in doc.tables:
    if not table._element.xpath('.//w:tblGrid'):
        tblGrid = OxmlElement('w:tblGrid')
        table._element.insert(0, tblGrid)

In [207]:
# 获取表格
table = doc.tables[0]

In [208]:
# 将表格转换为DataFrame
data = []
for i, row in enumerate(table.rows):
    text = [cell.text for cell in row.cells]
    data.append(text)

In [209]:
from typing import Literal, Optional


class base_option:
    def __init__(
        self,
        option_num: Literal[1, 2, 3, 4, 5] = 1,
        text: Optional[str] = None,
        is_bold: bool = None,
        is_italic: bool = None,
        FontColor: bool = None,
        HighlightColor: bool = None,
    ):
        self.option_num = option_num
        if text == None:
            text = str(option_num)
        self.text = text
        self.is_bold = is_bold
        self.is_italic = is_italic
        self.FontColor = FontColor
        self.HighlightColor = HighlightColor

    def __repr__(self) -> str:
        return f"text : {self.text} \nis_bold: {self.is_bold} \nis_italic: {self.is_italic} \nFontColor:{self.FontColor} \nHighlightColor:{self.HighlightColor}"

    def compare(self, option_num: Optional[int] = None,debug = False) -> bool:
        if option_num == None:
            option_num = self.option_num
        _base_option = base_option(option_num)
        if debug:
            print(f"self: {self}")
            print(f"_base_option: {_base_option}")
        for key in [
            "option_num",
            "text",
            "is_bold",
            "is_italic",
            "FontColor",
            "HighlightColor",
        ]:
            if getattr(self, key) != getattr(_base_option, key):
                return False
        return True


def get_option(run: docx.text.run.Run, option_num: Literal[1, 2, 3, 4, 5]):
    # 获取文本
    text = run.text
    # 获取文本的格式
    is_bold = run.bold
    is_italic = run.italic
    FontColor = run.font.color.rgb if run.font.color else None
    HighlightColor = run.font.highlight_color if run.font.highlight_color else None
    return base_option(option_num, text, is_bold, is_italic, FontColor, HighlightColor)

In [218]:
# 获取文本格式
for i in range(2, 6):
    row = table.rows[i]
    print(f"问题为: {row.cells[0].text}")
    for idx, cell in enumerate(row.cells[-5:]):
        if not get_option(cell.paragraphs[0].runs[0], idx + 1).compare():
            print(f"所选择的选项为: {idx+1}")

问题为: 如果我在这个团队犯了错误，这通常会被记在我的头上
所选择的选项为: 1
问题为: 这个团队的成员能够提出问题和棘手的议题
所选择的选项为: 2
问题为: 这个团队上的人不会因为别人的不同而排斥他们
所选择的选项为: 1
问题为: 在这个团队中冒险是安全的
所选择的选项为: 3
